In [1]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install keras
!pip install natsort
!pip install scikit-learn
!pip install Pillow

In [4]:
!pip install keras.layers

ERROR: Could not find a version that satisfies the requirement keras.layers (from versions: none)
ERROR: No matching distribution found for keras.layers


In [ ]:
!git clone https://github.com/yilangpeng/transfer-learning-clustering.git
!git clone https://github.com/GKalliatakis/Keras-VGG16-places365.git

Cloning into 'transfer-learning-clustering'...
remote: Enumerating objects: 1033, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1033 (delta 5), reused 0 (delta 0), pack-reused 1021
Receiving objects: 100% (1033/1033), 72.42 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Cloning into 'vgg16_hybrid_places_1365'...
fatal: could not read Username for 'https://github.com': No such device or address
Cloning into 'Keras-VGG16-places365'...
remote: Enumerating objects: 172, done.
remote: Total 172 (delta 0), reused 0 (delta 0), pack-reused 172
Receiving objects: 100% (172/172), 443.99 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [5]:
import os
import keras
import numpy as np
import pandas as pd
import natsort
import sklearn
from PIL import Image



In [4]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null


In [7]:
#dataset cleaning

def get_image_filenames(folder):
  image_filenames = []
  for filename in os.listdir(folder):
    if filename.lower().endswith(".jpg"):
      image_filenames.append(filename)
  return image_filenames

def write_filenames_to_txt(image_filenames, output_file):
    with open(output_file, 'w') as file:
        for filename in image_filenames:
            file.write(f"{filename}\n")


folder = "/content/drive/MyDrive/POLI179Project/img_all"

output_file = "/content/drive/MyDrive/POLI179Project/imgname2.txt"

image_filenames = get_image_filenames(folder)

write_filenames_to_txt(image_filenames, output_file)

In [8]:
print(image_filenames)


['amyklobuchar_2020-03-16134642EDT_IG.jpg', 'berniesanders_2019-10-15151544EDT_IG.jpg', 'berniesanders_2019-03-15195851EDT_IG.jpg', 'berniesanders_2019-11-24101732EST_IG.jpg', 'berniesanders_2019-11-20103701EST_IG.jpg', 'berniesanders_2019-09-16163410EDT_IG.jpg', 'berniesanders_2019-09-27150722EDT_IG.jpg', 'amyklobuchar_2019-04-26120041EDT_IG.jpg', 'berniesanders_2019-09-29185640EDT_IG.jpg', 'berniesanders_2019-10-31183304EDT_IG.jpg', 'berniesanders_2019-11-09203406EST_IG.jpg', 'berniesanders_2019-11-02213026EDT_IG.jpg', 'amyklobuchar_2020-02-09210342EST_IG.jpg', 'berniesanders_2019-11-21200555EST_IG.jpg', 'berniesanders_2019-12-12142815EST_IG.jpg', 'amyklobuchar_2019-02-25000537EST_IG.jpg', 'berniesanders_2019-04-16170747EDT_IG.jpg', 'berniesanders_2019-12-01110747EST_IG.jpg', 'berniesanders_2019-10-03215320EDT_IG.jpg', 'berniesanders_2019-04-13151233EDT_IG.jpg', 'berniesanders_2019-03-30181843EDT_IG.jpg', 'amyklobuchar_2019-09-30194303EDT_IG.jpg', 'amyklobuchar_2019-01-06165528EST_IG

In [9]:
import os, shutil

def create_path(filepath):
    filepathfolder = os.path.dirname(filepath)
    if not os.path.exists(filepathfolder): os.makedirs(filepathfolder)

def copy_file(filepath1, filepath2):
    filepathfolder2 = os.path.dirname(filepath2)
    if not os.path.exists(filepathfolder2): os.makedirs(filepathfolder2)
    shutil.copy2(filepath1, filepath2)

In [10]:
! cp /content/drive/MyDrive/POLI179Project/transfer-learning-clustering-master/transfer-learning-clustering-master/vgg16_hybrid_places_1365.py /content/vgg16_hybrid_places_1365.py

In [12]:
! cp /content/drive/MyDrive/POLI179Project/transfer-learning-clustering-master/transfer-learning-clustering-master/ypoften.py /content/ypoften.py

In [11]:
! cp /content/drive/MyDrive/POLI179Project/Keras-VGG16-places365-master/Keras-VGG16-places365-master/vgg16_hybrid_places_1365.py /content/vgg16_hybrid_places_1365.py

In [27]:
!pip install keras_applications

In [35]:
#Step 1 Feature Extraction
import os, joblib
import numpy as np
from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
import ypoften as of
from keras.layers import Dense, Activation, Dropout


cvmodel = "vgg16"
cvmodel = "vgg16hybrid"

if cvmodel == "vgg16":
    from keras.applications.vgg16 import VGG16
    base_model = VGG16(weights="imagenet", include_top=True)
    print(base_model.summary())
    feature_model = Model(base_model.input, base_model.get_layer('fc1').output)
    img_size = (224, 224)

elif cvmodel == "vgg16hybrid":
    #vgg16_hybrid_places_1365 = open("/content/drive/MyDrive/POLI179Project/transfer-learning-clustering-master/transfer-learning-clustering-master/vgg16_hybrid_places_1365.py"
    from vgg16_hybrid_places_1365 import VGG16_Hybrid_1365
    #vgg16_hybrid_places_13
    base_model = VGG16_Hybrid_1365(weights='places', include_top=True, input_shape=(224, 224, 3))
    print(base_model.summary())
    feature_model = Model(base_model.input, base_model.get_layer('fc2').output)
    img_size = (224, 224)

print(feature_model.summary())

cwd = os.path.join("/content/drive/MyDrive/POLI179Project")
# replace with your own folder path
imgnamefile = os.path.join(cwd,"imgname2.txt")
imgnamefile = open(imgnamefile, "r")
exfolder = os.path.join(cwd,"img exfeature", "")

for j, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip('\n\r')
    print(j, imgname)

    imgpath = os.path.join(cwd, 'img_all', imgname)
    img = image.load_img(imgpath, target_size=img_size) # read the image
    image_array = image.img_to_array(img) # convert the image to a numpy array

    image_expand = np.expand_dims(image_array, 0)
    x_train = preprocess_input(image_expand) # normalize the image to 0-1 range
    features_x = feature_model.predict(x_train) # extract features for each image

    featuresavepath = os.path.join(exfolder, cvmodel, imgname+".dat")
    of.create_path(featuresavepath)
    joblib.dump(features_x[0], featuresavepath) # save the extracted features
    print("==SUCCESS=="*20)

print("DONE"*20)

Model: "vgg16-hybrid1365"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 139,852,949 (533.50 MB)

 Trainable params: 139,852,949 (533.50 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

Streaming output truncated to the last 5000 lines.
==SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS==
2352 joebiden_2019-08-24162238EDT_IG.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
==SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS==
2353 joebiden_2019-08-24193913EDT_IG.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
==SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS==
2354 joebiden_2019-10-03211958EDT_IG.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
==SUCCESS====SUCCESS====SUCCESS====SUCCESS====SUCCESS==

In [37]:
#STep 2 Combine Features
cwd = os.path.join("/content/drive/MyDrive/POLI179Project") # replace with your own folder path
cvmodel = "vgg16hybrid"

imgnamefile = os.path.join(cwd,"imgname2.txt")
imgnamefile = open(imgnamefile, "r")
exfolder = os.path.join(cwd,"img exfeature", "")
features_array = []

for lineindex, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip('\r\n')
    ex_feature_path = os.path.join(cwd, 'img exfeature', cvmodel, imgname + ".dat")
    imgexfeatures = joblib.load(ex_feature_path)
    flatten_features = np.ndarray.flatten(imgexfeatures)
    features_array.append(flatten_features)

features_array = np.array(features_array)
features_savepath = os.path.join(cwd, 'img exfeature','features combine', cvmodel + ".dat")
of.create_path(features_savepath)
joblib.dump(features_array,features_savepath)

print("DONE"*20)

DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE


In [41]:
#Step 3 PCA
cwd = os.path.join("/content/drive/MyDrive/POLI179Project") # replace with your own folder path
cvmodel = "vgg16hybrid"

imgnamefile = os.path.join(cwd,"imgname2.txt")
imgnamefile = open(imgnamefile, "r")
exfolder = os.path.join(cwd,"img exfeature", "")

features_savepath = os.path.join(cwd,'img exfeature','features combine',cvmodel+'.dat')

features_array = joblib.load(features_savepath)
x = pd.DataFrame(features_array)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# standardizing the features
x = StandardScaler().fit_transform(x)

# PCA
pca = PCA(0.95) #95% of the variance is retained.
pca.fit(x)
x_pca = pca.transform(x)

# save PCA
savefolder = os.path.join(cwd,cwd,'img exfeature','features PCA')
pca_savepath = os.path.join(savefolder, cvmodel+'.dat')
of.create_path(pca_savepath)
joblib.dump(x_pca, pca_savepath)

# save components
components = pca.components_
savepath = os.path.join(savefolder, cvmodel + 'components.txt')
components = pd.DataFrame(components)
components.to_csv(savepath, header=None, index=None, sep='\t', mode='a')

# save variance explained ratio
variance = pca.explained_variance_ratio_
savepath = os.path.join(savefolder, cvmodel + ' variance.txt')
variance = pd.DataFrame(variance)
variance.to_csv(savepath, header=None, index=None, sep='\t', mode='a')

print("DONE"*20)

DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE


In [52]:
#Step 4 K means Clustering
cwd = os.path.join("/content/drive/MyDrive/POLI179Project") # replace with your own folder path
cvmodel = "vgg16hybrid"
clmethod = "KMeans"

from sklearn.cluster import KMeans
import joblib
import pandas as pd

features_savepath = os.path.join(cwd,'img exfeature','features PCA',cvmodel+'.dat')
features_array = joblib.load(features_savepath)
X = pd.DataFrame(features_array)
print(X)

nd = 200
X = X.iloc[:,0:nd]

savefolder = cvmodel + ' ' + clmethod + ' PCA' + str(nd)

for K in range(5, 21):
    print('number of cluster', K)
    cl = KMeans(K, random_state=0)
    cl.fit(X)
    labels = cl.labels_

    imgnamefile = os.path.join(cwd,"imgname2.txt")
    df = pd.read_csv(imgnamefile, sep ='\t', header = None)
    df.columns = ['imgname2']
    print(df)

    df['label'] = labels
    filepath = os.path.join(cwd,'img cluster',savefolder,str(K),'label.txt')
    of.create_path(filepath)
    df.to_csv(filepath, index = None, header = None, sep = '\t')

print("DONE"*20)

           0          1          2          3          4          5     \
0     -5.246964  -1.495325 -21.460293  -3.341855   2.709868  -8.122070   
1     17.600805   3.250836   0.376421  19.370430   9.006392  -0.339176   
2     10.422276  -1.713681  10.222207   3.013706   8.776044   9.614964   
3     35.745022  -4.149613   4.447752  -0.581134   7.599938  22.962448   
4     36.536255   2.927214   5.390571 -18.838593  -8.647396  -9.721010   
...         ...        ...        ...        ...        ...        ...   
4013  16.718563  -4.653560 -12.165168   1.288636  -3.203064   2.491545   
4014   8.714463  -3.200783 -14.364199   3.370840   2.982210  12.654929   
4015 -13.466227  18.776979  25.523605  -5.509771   9.846994   4.686201   
4016 -12.275115 -25.641954  10.604620  -1.883515 -10.065022   1.557327   
4017 -12.667787   9.006012 -14.367384  -7.448712  -1.439315   2.413440   

           6          7          8          9     ...      1282      1283  \
0     -3.228155  -1.516068   2.255

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                                        imgname2
0        amyklobuchar_2020-03-16134642EDT_IG.jpg
1       berniesanders_2019-10-15151544EDT_IG.jpg
2       berniesanders_2019-03-15195851EDT_IG.jpg
3       berniesanders_2019-11-24101732EST_IG.jpg
4       berniesanders_2019-11-20103701EST_IG.jpg
...                                          ...
4013  realdonaldtrump_2019-02-06171453EST_IG.jpg
4014  realdonaldtrump_2019-08-11210954EDT_IG.jpg
4015  realdonaldtrump_2019-02-01151415EST_IG.jpg
4016  realdonaldtrump_2019-08-15213437EDT_IG.jpg
4017  realdonaldtrump_2019-08-07165410EDT_IG.jpg

[4018 rows x 1 columns]
number of cluster 6


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                                        imgname2
0        amyklobuchar_2020-03-16134642EDT_IG.jpg
1       berniesanders_2019-10-15151544EDT_IG.jpg
2       berniesanders_2019-03-15195851EDT_IG.jpg
3       berniesanders_2019-11-24101732EST_IG.jpg
4       berniesanders_2019-11-20103701EST_IG.jpg
...                                          ...
4013  realdonaldtrump_2019-02-06171453EST_IG.jpg
4014  realdonaldtrump_2019-08-11210954EDT_IG.jpg
4015  realdonaldtrump_2019-02-01151415EST_IG.jpg
4016  realdonaldtrump_2019-08-15213437EDT_IG.jpg
4017  realdonaldtrump_2019-08-07165410EDT_IG.jpg

[4018 rows x 1 columns]
number of cluster 7


...

                                        imgname2
0        amyklobuchar_2020-03-16134642EDT_IG.jpg
1       berniesanders_2019-10-15151544EDT_IG.jpg
2       berniesanders_2019-03-15195851EDT_IG.jpg
3       berniesanders_2019-11-24101732EST_IG.jpg
4       berniesanders_2019-11-20103701EST_IG.jpg
...                                          ...
4013  realdonaldtrump_2019-02-06171453EST_IG.jpg
4014  realdonaldtrump_2019-08-11210954EDT_IG.jpg
4015  realdonaldtrump_2019-02-01151415EST_IG.jpg
4016  realdonaldtrump_2019-08-15213437EDT_IG.jpg
4017  realdonaldtrump_2019-08-07165410EDT_IG.jpg

[4018 rows x 1 columns]
number of cluster 19


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                                        imgname2
0        amyklobuchar_2020-03-16134642EDT_IG.jpg
1       berniesanders_2019-10-15151544EDT_IG.jpg
2       berniesanders_2019-03-15195851EDT_IG.jpg
3       berniesanders_2019-11-24101732EST_IG.jpg
4       berniesanders_2019-11-20103701EST_IG.jpg
...                                          ...
4013  realdonaldtrump_2019-02-06171453EST_IG.jpg
4014  realdonaldtrump_2019-08-11210954EDT_IG.jpg
4015  realdonaldtrump_2019-02-01151415EST_IG.jpg
4016  realdonaldtrump_2019-08-15213437EDT_IG.jpg
4017  realdonaldtrump_2019-08-07165410EDT_IG.jpg

[4018 rows x 1 columns]
number of cluster 20


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                                        imgname2
0        amyklobuchar_2020-03-16134642EDT_IG.jpg
1       berniesanders_2019-10-15151544EDT_IG.jpg
2       berniesanders_2019-03-15195851EDT_IG.jpg
3       berniesanders_2019-11-24101732EST_IG.jpg
4       berniesanders_2019-11-20103701EST_IG.jpg
...                                          ...
4013  realdonaldtrump_2019-02-06171453EST_IG.jpg
4014  realdonaldtrump_2019-08-11210954EDT_IG.jpg
4015  realdonaldtrump_2019-02-01151415EST_IG.jpg
4016  realdonaldtrump_2019-08-15213437EDT_IG.jpg
4017  realdonaldtrump_2019-08-07165410EDT_IG.jpg

[4018 rows x 1 columns]
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE


In [53]:
#Step 5 Copy Image
cwd = os.path.join("/content/drive/MyDrive/POLI179Project") # replace with your own folder path
cvmodel = "vgg16hybrid"
clmethod = "KMeans"

savefolder = "vgg16hybrid KMeans PCA200"

def save_img(random_id, imgname, label, K):
    imgpath1=os.path.join(cwd,'img_all',imgname)
    newimgname = str(random_id)+' '+imgname
    imgpath2 = os.path.join(cwd,'img cluster',savefolder, str(K),str(label),newimgname)
    of.copy_file(imgpath1, imgpath2)

for K in range(5, 21):
    print('number of cluster',K)
    filepath = os.path.join(cwd,'img cluster',savefolder,str(K),'label.txt')
    df = pd.read_csv(filepath, sep ='\t', header = None)
    df.columns = ['imgname',"label"]

    dr = df.sample(frac=1, random_state=42) # random shuffle images
    dr['random_id'] = np.arange(len(dr)) # add random id

    for labelK in range(0, K):
        dc = dr.loc[dr['label'] == labelK] # select images from each cluster
        dc = dc.iloc[:20,:] # select 20 images from each cluster
        dc.apply(lambda row: save_img(row['random_id'],row['imgname'],row['label'], K),axis=1)

    print("DONE"*20)
print("DONE"*20)

number of cluster 5
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 6
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 7
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 8
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 9
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 10
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 11
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 12
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 13
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONE
number of cluster 14
DONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDONEDO

In [54]:
#Step 6 Visualize Grid

from PIL import Image, ImageDraw, ImageFont
import os
import glob
from natsort import natsorted
def fill_square(im, tbw):
    size = (tbw, tbw)
    bg = Image.new('RGB', size, "white")  # create a background image
    im.thumbnail(size, Image.LANCZOS)
    w, h = im.size
    bg.paste(im, (int((size[0]-w)/2), int((size[1]-h)/2)))
    return(bg)

cwd = os.path.join("/content/drive/MyDrive/POLI179Project") # replace with your own folder path
cvmodel = "vgg16hybrid"
clmethod = "KMeans"

savefolder = "vgg16hybrid KMeans PCA200"

w_sq = 300 # width/height of image
w_clusterid = 60 # width of text for cluster id
h_imgid = 60 # height of gap
fntcluster = ImageFont.load_default()

for K in range(5, 21):
    ncol = 20 # how many images to show in each row
    nrow = 1
    nimg = ncol * nrow # total number of image in each cluster to show
    large_w = w_sq*ncol + w_clusterid # total width of the entire image
    large_h = (w_sq + h_imgid) * nrow * K # total height of the entire image
    large = Image.new('RGB', (large_w, large_h), "white")

    for label in range(0, K):
        print('-'* 10, K, label)
        imgpathfolder = os.path.join(cwd,'img cluster',savefolder,str(K),str(label),'')
        imgfiles = glob.glob(imgpathfolder + "*.jpg")
        imgfiles = natsorted(imgfiles)[:nimg]

        d = ImageDraw.Draw(large)
        d.text((0, label * nrow * (w_sq + h_imgid) + 15), str(label+1), font=fntcluster, fill=(0, 0, 0))

        for j, imgfile in enumerate(imgfiles):
            jw = j%ncol; jh = j//ncol
            img = Image.open(imgfile)
            im_sq = fill_square(img, w_sq)
            im_sq_x = jw*w_sq + w_clusterid
            im_sq_y = h_imgid + jh* (w_sq + h_imgid) + label * nrow * (w_sq + h_imgid)
            large.paste(im_sq, (im_sq_x, im_sq_y))

    large_savepath = os.path.join(cwd, "img cluster", savefolder, 'grid', str(K)+".png")
    of.create_path(large_savepath)
    large.save(large_savepath)

print("DONE"*20)

---------- 5 0
---------- 5 1
---------- 5 2
---------- 5 3
---------- 5 4
---------- 6 0
---------- 6 1
---------- 6 2
---------- 6 3
---------- 6 4
---------- 6 5
---------- 7 0
---------- 7 1
---------- 7 2
---------- 7 3
---------- 7 4
---------- 7 5
---------- 7 6
---------- 8 0
---------- 8 1
---------- 8 2
---------- 8 3
---------- 8 4
---------- 8 5
---------- 8 6
---------- 8 7
---------- 9 0
---------- 9 1
---------- 9 2
---------- 9 3
---------- 9 4
---------- 9 5
---------- 9 6
---------- 9 7
---------- 9 8
---------- 10 0
---------- 10 1
---------- 10 2
---------- 10 3
---------- 10 4
---------- 10 5
---------- 10 6
---------- 10 7
---------- 10 8
---------- 10 9
---------- 11 0
---------- 11 1
---------- 11 2
---------- 11 3
---------- 11 4
---------- 11 5
---------- 11 6
---------- 11 7
---------- 11 8
---------- 11 9
---------- 11 10
---------- 12 0
---------- 12 1
---------- 12 2
---------- 12 3
---------- 12 4
---------- 12 5
---------- 12 6
---------- 12 7
----------